# Assistant Log Extractor (para Watson Studio)

## Introdução
Este notebook auxilia a fazer a extração, manipulação e visualização dos dados de logs do IBM Watson Assistant no Watson Studio.

## 1. Configuração e setup

### 1.1 Instalação de bibliotecas necessárias

In [ ]:
import ibm_watson #autenticação na IBM Cloud
#from project_lib import Project #autenticação nos projetos do IBM Watson Studio
import csv
import pandas as pd
import extractor
from datetime import datetime
from time import sleep

### 1.2 Credenciais
Preencha as variáveis com as credenciais do Watson Assistant e do projeto do Watson Studio.

In [ ]:
assistant_apikey = 'APIKEY'
assistant_workspace_id = 'WORKSPACEID'
#studio_project_id = 'PROJETC_ID'
#studio_project_access_token = 'PROJECT_TOKEN'
#O token do projeto é um token de autorização para acessar recursos como dados e conexões.

### 1.3 Instância do Watson Assistant

In [ ]:
service = ibm_watson.AssistantV1(
            iam_apikey = assistant_apikey,
            version = '2019-02-28',
            url= 'https://gateway.watsonplatform.net/assistant/api'
)
workspace_id = assistant_workspace_id

### 1.4 Instância do Projeto no Watson Studio

In [ ]:
project = Project(project_id=studio_project_id, project_access_token=studio_project_access_token)
pc = project.project_context

## 2. Extração de logs
As próximas células definem o filtro para a extração dos logs. 
**⚠️ Execute apenas uma das células desta seção.**

### 2.1 Filtro de data
Insira as datas para o período dos logs que você deseja extrair, seguindo o formato de data.

In [ ]:
de = input("De (AAAA-MM-DD): ")
ate = input("Até (AAAA-MM-DD): ")
filter_criteria = 'response_timestamp>='+ de + ',response_timestamp<=' + ate
print(filter_criteria)

### 2.2  Filtro de turno
Busca apenas mensagens de primeiro turno da conversa.

In [ ]:
filter_criteria = 'response.context.system.dialog_turn_counter::1'

### 2.3 Outros filtros
Preencha o campo de filtro conforme a query que deseja fazer. Consute [aqui](https://cloud.ibm.com/docs/services/assistant?topic=assistant-filter-reference) os filtros e formatos.

In [ ]:
filter_criteria = ''

### 2.4 Extração de logs


In [ ]:
log_extraction = service.list_logs(workspace_id, filter=filter_criteria, page_limit=1000).get_result() 
#chamada para recupear o log

## 3. Construção da tabela

### 3.1 Criação da tabela

In [ ]:
print(datetime.now())
table = pd.DataFrame(columns=["conversation_id",
                        "turn_counter",
                        "date",
                        "time",
                        "input", 
                        "intent",
                        "confidence", 
                        "entities:value",
                        "bot_message", 
                        "log_id"]) #adiciona as colunas da tabela
counter = 0
while True:
    for element in log_extraction["logs"]:
        line = extractor.extractor_list(element) #chama a função que organiza o .json em linhas para tabela
        table = table.append(pd.Series(line, index=table.columns), ignore_index=True)
    counter += 1
    print("Página {} processada".format(str(counter)))
    if counter >= 24:
        print("Limite de request excedido. Aguardando 60 segundos para retomar.")
        sleep(60)
        counter = 0
    if 'next_url' in log_extraction['pagination'].keys(): #confere paginação
        cursor = log_extraction['pagination']['next_cursor']
        log_extraction = service.list_logs(
            workspace_id, 
            filter=filter_criteria, 
            page_limit=1000, 
            cursor=cursor).get_result()
    else:
        break
print(datetime.now())

### 3.2 Visualização da tabela

In [ ]:
table

### 3.3 Salvar tabela WS

In [ ]:
extraction_date = datetime.strftime(datetime.now(), "%Y-%m-%d")

project.save_data(extraction_date + ".csv", table.to_csv(), overwrite=True) 
#salva a planilha em .csv no ObjectStorage da IBM

### 3.4 Salvar tabela local

In [ ]:
table.to_csv(str(datetime.today()) + filter_criteria + ".csv", index=False)
#salva a planilha em .csv no diretório local

## 4. Manipulação de dados

In [ ]:
table.sort_values(by=['date','conversation_id','turn_counter'], inplace=True, ascending=True)
#ordena a tabela conforme data e hora, id da conversa e turno, do menor para o maior
table

In [ ]:
intent_count_table = table.groupby(by=['intent'])['intent'].count()
confidence_mean_table = table.groupby(by=['intent'])['confidence'].mean()

intent_count_table

In [ ]:
confidence_mean_table

In [ ]:
intent_table = pd.DataFrame(columns=["intents", "occurances", "confidence_mean"])
for i in range(len(intent_count_table.keys())):
    intent = intent_count_table.keys()[i]
    occurance = intent_count_table[i]
    confidence = confidence_mean_table[i]
    line = [intent, occurance, confidence]
    intent_table = intent_table.append(pd.Series(line, index=intent_table.columns), ignore_index=True)
intent_table    

In [ ]:
messages_no = table['log_id'].nunique()
conversations_no = table['conversation_id'].nunique()

print('''=====REPORT====
Nº de mensagens: {}
Nº de conversas: {}
Média de mensagens: {}
'''.format(messages_no, conversations_no, messages_no/conversations_no))